In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
df_train = pd.read_csv("satires.csv")
df_train.sample(10)

,title,text,label
1815,ter duas segundasfeiras na mesma semana e exp...,a semana comecou como sempre com uma segundafe...,satire
1380,cunha pede que dilma ache seu par de meia que...,depois de demitir o ministro da educacao o pre...,satire
7141,lula ficou preocupado com declaracao de genera...,o expresidente luiz inacio lula da silva disse...,real
2483,cardiologistas recomendam check up antes de c...,o ano comeca com as mesmas despesas de sempre ...,satire
2936,aluno nota zero no enem desabafa no facebook,o aluno j. h. 16 anos nao aceitou bem a nota ...,satire
8646,nao falo de processos diz cunha sobre intimaca...,o deputado cassado eduardo cunha pmdbrj a prin...,real
2338,eleitor aperta 45 na urna e ve malafaia com t...,o eleitor joacyr monteiro dos santos foi votar...,satire
3159,dona de casa passa roupa com ferro desligado ...,a dona de casa amelia dos santos e silva foi c...,satire
7207,operacao boca livre procura 14 por fraudes de ...,a policia federal deflagrou em conjunto com a ...,real
9488,na falta de confianca cliente pode revogar con...,mesmo existindo clausula de irrevogabilidade d...,real


In [71]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   10000 non-null  object
 1   text    10000 non-null  object
 2   label   10000 non-null  object
dtypes: object(3)
memory usage: 234.5+ KB


In [72]:
df_train['label'].value_counts()


satire    5000
real      5000
Name: label, dtype: int64

In [73]:
df_satires = df_train.rename(columns={'label': 'is_satire'})

In [74]:
df_satires['is_satire'].replace({"satire": int(1), "real": int(0)}, inplace=True)
df_satires = df_satires.dropna()

In [75]:
df_satires['title_text'] = df_satires['title'] + " " + df_satires['text']
df_satires

,title,text,is_satire,title_text
0,crise e tao grande que nem tiozao do pave fez...,a familia guimaraes passou a noite de natal pe...,1,crise e tao grande que nem tiozao do pave fez...
1,nao me representam diz jesus sobre intolerant...,uma menina de 11 anos apedrejada ao sair de um...,1,nao me representam diz jesus sobre intolerant...
2,marina silva e heloisa helena montam novo par...,insatisfeitas com seus partidos com as siglas ...,1,marina silva e heloisa helena montam novo par...
3,dez propostas que podem realmente mudar o brasil,o instituto nupal nucleo de pesquisas da ameri...,1,dez propostas que podem realmente mudar o bra...
4,apresentadora do cidade alerta bahia dara cur...,assassinatos sequestros mortes violentas. nen...,1,apresentadora do cidade alerta bahia dara cur...
...,...,...,...,...
9995,filho de jader barbalho omite r 434 mil na jus...,o candidato a governador do para helder barbal...,0,filho de jader barbalho omite r 434 mil na jus...
9996,aecio diz a fachin que vai fechar escritorio p...,o senador aecio neves psdbmg por meio de sua d...,0,aecio diz a fachin que vai fechar escritorio p...
9997,brincando com fogo,se hoje o governo de michel temer nao esta env...,0,brincando com fogo se hoje o governo de michel...
9998,nao tinha ninguem na camara municipal,o brasil a despeito de outros paises que nao m...,0,nao tinha ninguem na camara municipal o brasil...


In [76]:
stop_words = set(stopwords.words("portuguese"))

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv_title = CountVectorizer(stop_words=stop_words, ngram_range = (1,1),tokenizer = token.tokenize)
title_counts = cv_title.fit_transform(df_satires['title'])

cv_text = CountVectorizer(stop_words=stop_words, ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv_text.fit_transform(df_satires['text'])

cv_tt = CountVectorizer(stop_words=stop_words, ngram_range = (1,1),tokenizer = token.tokenize)
tt_counts = cv_tt.fit_transform(df_satires['title_text'])


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['amos', 'at', 'est', 'estiv', 'f', 'h', 'houv', 'j', 'm', 'n', 'nhamos', 'ramos', 's', 'ser', 'ssemos', 't', 'tamb', 'ter', 'tiv', 'vamos', 'voc'] not in stop_words.
  % sorted(inconsistent)


#### Predição a partir dos títulos


In [78]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(title_counts, df_satires['is_satire'], test_size=0.25, random_state=5)

In [79]:
from sklearn.naive_bayes import MultinomialNB
MNB1 = MultinomialNB()
MNB1.fit(X_train1, Y_train1)

MultinomialNB()

In [80]:
from sklearn import metrics
predicted_nb1 = MNB1.predict(X_test1)
accuracy_score_nb1 = metrics.accuracy_score(predicted_nb1, Y_test1)

In [81]:
print("Acurácia do Naive Bayes a partir dos títulos: " + str('{:04.2f}'.format(accuracy_score_nb1*100))+'%')

Acurácia do Naive Bayes a partir dos títulos: 86.96%


In [82]:
from sklearn.linear_model import LogisticRegression

In [83]:
logistic_regression1 = LogisticRegression(random_state=0).fit(X_train1, Y_train1)

In [84]:
predicted_lg1 = logistic_regression1.predict(X_test1)
LG_accuracy_score1 = metrics.accuracy_score(predicted_lg1, Y_test1)

print("Acurácia do Logistic Regression a partir dos títulos: " + str('{:04.2f}'.format(LG_accuracy_score1*100))+'%')

Acurácia do Logistic Regression a partir dos títulos: 87.68%


#### Predição a partir dos textos


In [85]:
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(text_counts, df_satires['is_satire'], test_size=0.25, random_state=5)

In [86]:
MNB2 = MultinomialNB()
MNB2.fit(X_train2, Y_train2)

MultinomialNB()

In [87]:
predicted_nb2 = MNB2.predict(X_test2)
accuracy_score_nb2 = metrics.accuracy_score(predicted_nb2, Y_test2)

In [88]:
print("Acurácia do Naive Bayes a partir dos textos: " + str('{:04.2f}'.format(accuracy_score_nb2*100))+'%')

Acurácia do Naive Bayes a partir dos textos: 95.08%


In [89]:
logistic_regression2 = LogisticRegression(random_state=0).fit(X_train2, Y_train2)

In [90]:
predicted_lg2 = logistic_regression2.predict(X_test2)
LG_accuracy_score2 = metrics.accuracy_score(predicted_lg2, Y_test2)

In [91]:
print("Acurácia do Logistic Regression a partir dos textos: " + str('{:04.2f}'.format(LG_accuracy_score2*100))+'%')

Acurácia do Logistic Regression a partir dos textos: 98.16%


#### Predição a partir dos títulos concatenados com os textos

In [92]:
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(tt_counts, df_satires['is_satire'], test_size=0.25, random_state=5)

In [93]:
MNB3 = MultinomialNB()
MNB3.fit(X_train3, Y_train3)

MultinomialNB()

In [94]:
predicted_nb3 = MNB3.predict(X_test3)
accuracy_score_nb3 = metrics.accuracy_score(predicted_nb3, Y_test3)

In [95]:
print("Acurácia do Naive Bayes a partir dos títulos + textos: " + str('{:04.2f}'.format(accuracy_score_nb3*100))+'%')

Acurácia do Naive Bayes a partir dos títulos + textos: 95.40%


In [96]:
logistic_regression3 = LogisticRegression(random_state=0).fit(X_train3, Y_train3)

In [97]:
predicted_lg3 = logistic_regression3.predict(X_test3)
LG_accuracy_score3 = metrics.accuracy_score(predicted_lg3, Y_test3)

In [98]:
print("Acurácia do Logistic Regression a partir dos títulos + textos: " + str('{:04.2f}'.format(LG_accuracy_score3*100))+'%')

Acurácia do Logistic Regression a partir dos títulos + textos: 98.20%
